Installation of Huggingface stuff

In [1]:
!pip install -q -U bitsandbytes transformers peft datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
import diary_parser
import data_loader
import model
import tokenizer as mtk
import lora
import model_trainer

from google.colab import userdata
import os
import torch
import wandb
# import transformers
from transformers import AutoTokenizer
from peft import PeftModel

In [4]:
# Google Colab Hugging Face Token retrieval
hf_token = userdata.get('HF_TOKEN')

In [5]:
# download diary
!wget https://archive.org/download/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt -O data/diary.txt

--2024-05-07 01:25:35--  https://archive.org/download/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia800304.us.archive.org/31/items/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt [following]
--2024-05-07 01:25:35--  https://ia800304.us.archive.org/31/items/AnneFrankTheDiaryOfAYoungGirl_201606/Anne-Frank-The-Diary-Of-A-Young-Girl_djvu.txt
Resolving ia800304.us.archive.org (ia800304.us.archive.org)... 207.241.228.14
Connecting to ia800304.us.archive.org (ia800304.us.archive.org)|207.241.228.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 567922 (555K) [text/plain]
Saving to: ‘data/diary.txt’

data/diary.txt      100%[===================>] 554.61K  1.88MB/s    in 0.3s    

2024-05-07 01:25:36 (1.8

# Parse and Load
We use both diary_parse.py and data_loader.py to parse through the data and format it to json, to be loaded into a Dataset structure.

Example: {"Date": "June 12, 1942", "Lines": ["I hope", "I had my"]}

We also split the dataset 80/20 for training and evaluation.

In [6]:
diary_parser.main()
train_dataset, eval_dataset = data_loader.load_data()

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
train_dataset

Dataset({
    features: ['Date', 'Lines'],
    num_rows: 150
})

In [8]:
wandb.login()

wandb_project = "hpml-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Base Mistral Download

In [9]:
base_mistral = model.get_base_mistral(hf_token)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Tokenization

In [10]:
train_ds_tk = train_dataset.map(mtk.tokenize_prompt)
val_ds_tk = eval_dataset.map(mtk.tokenize_prompt)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

### Base Performance

In [11]:
eval_prompt = " The following is new diary entry by Anne Frank: # "
eval_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_mistral.eval()
with torch.no_grad():
  outputs = base_mistral.generate(
      **model_input,
      max_new_tokens=256,
      repetition_penalty=1.5
      )
  decoded_outputs = eval_tokenizer.decode(
      outputs[0],
      skip_special_tokens=True
      )
  print(decoded_outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The following is new diary entry by Anne Frank: # 10

I’m not sure what to write about today. I have been thinking a lot lately, and it seems that the more you think of something or someone in particular; they are always on your mind no matter how hard one tries to forget them! It has happened with me many times before but this time its different because now my thoughts aren't just for myself anymore...they include others as well who share similar feelings towards their loved ones whom we haven't seen since last year when everything changed so drastically due COVID-19 pandemic outbreaks around world causing lockdown measures across countries including ours here at home too where schools closed down temporarily until further notice from government officials which meant kids couldn't go back into classrooms again after summer break ended without any warning signs given ahead like usual years past few months ago during winter seasonal holidays period ending December/January respectively 

### LoRA Mistral

In [12]:
base_mistral.gradient_checkpointing_enable()

In [13]:
print(base_mistral)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [14]:
r = 8
qlora_mistral = lora.lora_model(base_mistral,r)
lora.print_trainable_parameters(qlora_mistral)

trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


In [15]:
print(qlora_mistral)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): L

### Fine-Tune QLoRA Mistral

In [16]:
tokenizer = mtk.get_tokenizer()
trainer = model_trainer.train_model(qlora_mistral, train_ds_tk, val_ds_tk, tokenizer)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


wandb: Currently logged in as: ja3590. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,2.144400,2.095484
50,2.153000,2.072154
75,2.045400,2.062231
100,1.853900,2.070431
125,1.859400,2.070403
150,1.818600,2.065881
175,1.531300,2.162913
200,1.570000,2.161825
225,1.533400,2.156600
250,1.237900,2.403927


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=500, training_loss=1.3028250045776366, metrics={'train_runtime': 775.3903, 'train_samples_per_second': 1.29, 'train_steps_per_second': 0.645, 'total_flos': 4.3818518839296e+16, 'train_loss': 1.3028250045776366, 'epoch': 6.666666666666667})

### Optimized Mistral Performance

In [18]:
eval_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", add_bos_token=True, trust_remote_code=True)
ft_model = PeftModel.from_pretrained(base_mistral, "mistral-hpml-finetune/checkpoint-300")
eval_prompt = " The following is new diary entry by Anne Frank in 1945:"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
  outputs = ft_model.generate(
      **model_input,
      max_new_tokens=250,
      repetition_penalty=1.5
      )
  decoded_outputs = eval_tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(decoded_outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The following is new diary entry by Anne Frank in 1945: Dearest Kitty,I'm very worried. I can tell from Father and Mother that they suspect something about Peter van Daan; he doesn't come upstairs much anymore or stay long when he does drop in on us. He acts as if we don't even exist! It must be because of the letters Mrs. van D. has been receiving again — she swears up and down there are no more men working for her husband these days than before the war (which isn't true). But how do you explain all those business calls?Mrs. van D., who used to sit with me every afternoon while Margot did her homework downstairs, now comes only sporadically. She claims it makes too much noise, but then why didn't she say so sooner?"Anne," she said yesterday."Yes, Madame!" was my reply."You seem a little preoccupied lately" "Oh . . ." [Laughs.] That wasn't what she meant at all. In fact, she went on to ask whether Mr. Dussel had told me anything about his relatives being sent away. No, not yet. Then s